# AI21

[AI21 Studio](https://docs.ai21.com/) provides API access to `Jurassic-2` large language models.

This example goes over how to use LangChain to interact with [AI21 models](https://docs.ai21.com/docs/jurassic-2-models).

In [1]:
# install the package:
%pip install --upgrade --quiet  ai21

Note: you may need to restart the kernel to use updated packages.


In [1]:
# get AI21_API_KEY. Use https://studio.ai21.com/account/account

from getpass import getpass

AI21_API_KEY = getpass()

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import AI21

In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [5]:
llm = AI21(ai21_api_key=AI21_API_KEY)

In [6]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [8]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.invoke(question)

{'question': 'What NFL team won the Super Bowl in the year Justin Beiber was born?',
 'text': "\nThe Super Bowl is the annual championship game of the National Football League (NFL), a professional American football league.\n\nTake away NFL from the phrase and we are left with Bowl.\nBowl is a word used for trophies.\n\nSince there is no trophy called Super Bowl, let's try to find the team that won a Bowl in the year Justin Beiber was born.\n\nLet's find that out.\nIn the year 1994, the 27th of May, Justin Beiber was born.\n\nLet's find which team won a Bowl in the year 1994.\n\nResearching now.\n\nIn the year 1994, the 27th of May, 1994, the 68th edition of NFL championship was played, won by the Dallas Cowboys.\n\nFinal answer: Dallas Cowboys"}